In [1]:
import os

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    api_file: Path
    dataset: str


In [3]:
from cdclassifier.constants import *
from cdclassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            api_file=config.api_file,
            dataset=config.dataset
        ) 

        return data_ingestion_config
        

In [5]:
%cd ./chickenDiseaseClassification-mlproject/

[Errno 2] No such file or directory: './chickenDiseaseClassification-mlproject/'
/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject/research


In [8]:
%pwd

'/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject'

In [9]:
cm = ConfigurationManager()

[2024-08-18 03:50:43,839: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-18 03:50:43,840: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 03:50:43,842: INFO: common: created directory at: artifacts]


In [10]:
import os
import zipfile
from cdclassifier import logger
from cdclassifier.utils.common import get_size
from cdclassifier.utils.common import get_dataset

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # filename, header = request.urlretrieve(
            #     url=self.config.source_URL,
            #     filename = self.config.local_data_file
            # )
            get_dataset(
                secret_dir = Path(self.config.api_file),
                dataset= self.config.dataset,
                dataset_path= Path(self.config.local_data_file)
            )
            filename = self.config.local_data_file.split('/')[-1]
            
            logger.info(f"{filename} download complete!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-08-18 04:01:25,569: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-18 04:01:25,571: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 04:01:25,571: INFO: common: created directory at: artifacts]
[2024-08-18 04:01:25,572: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-18 04:01:25,572: INFO: 1304739324: File already exists of size: ~ 272257 KB]
